In [3]:
import os, glob
import random
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
import pandas as pd
import multiprocessing
from copy import deepcopy
from sklearn.metrics import precision_recall_curve, auc
import tensorflow.keras as keras

In [4]:
train_df = pd.read_csv('./input/train.csv')
train_df.head()

,Image_Label,EncodedPixels
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...
2,0011165.jpg_Gravel,NaN
3,0011165.jpg_Sugar,NaN
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...


In [5]:
train_df = train_df[~train_df['EncodedPixels'].isnull()]

In [6]:
train_df

,Image_Label,EncodedPixels
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...
5,002be4f.jpg_Flower,1339279 519 1340679 519 1342079 519 1343479 51...
7,002be4f.jpg_Sugar,67495 350 68895 350 70295 350 71695 350 73095 ...
8,0031ae9.jpg_Fish,3510 690 4910 690 6310 690 7710 690 9110 690 1...
9,0031ae9.jpg_Flower,2047 703 3447 703 4847 703 6247 703 7647 703 9...
11,0031ae9.jpg_Sugar,658170 388 659570 388 660970 388 662370 388 66...
13,0035239.jpg_Flower,100812 462 102212 462 103612 462 105012 462 10...
14,0035239.jpg_Gravel,65400 380 66800 380 68200 380 69600 380 71000 ...


In [7]:
train_df['Image'] = train_df['Image_Label'].map(lambda x: x.split('_')[0])
train_df['Class'] = train_df['Image_Label'].map(lambda x: x.split('_')[1])

In [8]:
train_df

,Image_Label,EncodedPixels,Image,Class
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...,0011165.jpg,Fish
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...,0011165.jpg,Flower
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...,002be4f.jpg,Fish
5,002be4f.jpg_Flower,1339279 519 1340679 519 1342079 519 1343479 51...,002be4f.jpg,Flower
7,002be4f.jpg_Sugar,67495 350 68895 350 70295 350 71695 350 73095 ...,002be4f.jpg,Sugar
8,0031ae9.jpg_Fish,3510 690 4910 690 6310 690 7710 690 9110 690 1...,0031ae9.jpg,Fish
9,0031ae9.jpg_Flower,2047 703 3447 703 4847 703 6247 703 7647 703 9...,0031ae9.jpg,Flower
11,0031ae9.jpg_Sugar,658170 388 659570 388 660970 388 662370 388 66...,0031ae9.jpg,Sugar
13,0035239.jpg_Flower,100812 462 102212 462 103612 462 105012 462 10...,0035239.jpg,Flower
14,0035239.jpg_Gravel,65400 380 66800 380 68200 380 69600 380 71000 ...,0035239.jpg,Gravel


In [9]:
classes = train_df['Class'].unique()

In [10]:
classes

array(['Fish', 'Flower', 'Sugar', 'Gravel'], dtype=object)

In [11]:
train_df = train_df.groupby('Image')['Class'].agg(set).reset_index()

In [12]:
train_df

,Image,Class
0,0011165.jpg,"{Fish, Flower}"
1,002be4f.jpg,"{Fish, Flower, Sugar}"
2,0031ae9.jpg,"{Fish, Flower, Sugar}"
3,0035239.jpg,"{Flower, Gravel}"
4,003994e.jpg,"{Fish, Sugar, Gravel}"
5,00498ec.jpg,{Gravel}
6,006bf7c.jpg,"{Fish, Sugar}"
7,006c5a6.jpg,"{Fish, Sugar}"
8,008233e.jpg,{Sugar}
9,008a5ff.jpg,"{Fish, Sugar}"


In [13]:
for class_name in classes:
    train_df[class_name] = train_df['Class'].map(lambda x: 1 if class_name in x else 0)
train_df.head()

,Image,Class,Fish,Flower,Sugar,Gravel
0,0011165.jpg,"{Fish, Flower}",1,1,0,0
1,002be4f.jpg,"{Fish, Flower, Sugar}",1,1,1,0
2,0031ae9.jpg,"{Fish, Flower, Sugar}",1,1,1,0
3,0035239.jpg,"{Flower, Gravel}",0,1,0,1
4,003994e.jpg,"{Fish, Sugar, Gravel}",1,0,1,1


In [14]:
img_2_ohe_vector = {img:vec for img, vec in zip(train_df['Image'], train_df.iloc[:, 2:].values)}

In [15]:
img_2_ohe_vector

{'0011165.jpg': array([1, 1, 0, 0]),
 '002be4f.jpg': array([1, 1, 1, 0]),
 '0031ae9.jpg': array([1, 1, 1, 0]),
 '0035239.jpg': array([0, 1, 0, 1]),
 '003994e.jpg': array([1, 0, 1, 1]),
 '00498ec.jpg': array([0, 0, 0, 1]),
 '006bf7c.jpg': array([1, 0, 1, 0]),
 '006c5a6.jpg': array([1, 0, 1, 0]),
 '008233e.jpg': array([0, 0, 1, 0]),
 '008a5ff.jpg': array([1, 0, 1, 0]),
 '0091591.jpg': array([0, 1, 1, 1]),
 '0095357.jpg': array([0, 0, 1, 0]),
 '009e2f3.jpg': array([1, 0, 1, 1]),
 '00a0954.jpg': array([0, 0, 1, 1]),
 '00b81e1.jpg': array([0, 1, 1, 1]),
 '00bea06.jpg': array([1, 0, 0, 1]),
 '00cedfa.jpg': array([0, 0, 1, 1]),
 '00d4443.jpg': array([0, 0, 1, 0]),
 '00dec6a.jpg': array([1, 1, 1, 1]),
 '0100a84.jpg': array([1, 0, 0, 1]),
 '0104b5b.jpg': array([0, 0, 1, 0]),
 '0107838.jpg': array([1, 0, 1, 1]),
 '0118bff.jpg': array([0, 1, 1, 0]),
 '011ba04.jpg': array([1, 1, 1, 0]),
 '01242d7.jpg': array([1, 0, 1, 1]),
 '014310a.jpg': array([1, 0, 1, 0]),
 '0146ef3.jpg': array([1, 0, 0, 0]),
 

In [16]:
train_imgs, val_imgs = train_test_split(train_df['Image'].values, 
                                        test_size=0.2, 
                                        stratify=train_df['Class'].map(lambda x: str(sorted(list(x)))), # sorting present classes in lexicographical order, just to be sure
                                        random_state=2019)

In [17]:
train_imgs

array(['a0e6825.jpg', '948a898.jpg', '4b67f58.jpg', ..., '42dca86.jpg',
       '551efa2.jpg', '74a109d.jpg'], dtype=object)

In [18]:
val_imgs

array(['cce531c.jpg', '3e5339e.jpg', '5127b0e.jpg', ..., '265e050.jpg',
       '9203dd2.jpg', '61aa8dc.jpg'], dtype=object)

In [46]:
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import Sequence
from albumentations import Compose, VerticalFlip, HorizontalFlip, Rotate, GridDistortion
import matplotlib.pyplot as plt
from IPython.display import Image
from tqdm import tqdm_notebook as tqdm
from numpy.random import seed
seed(10)
import tensorflow as tf
#from tensorflow import set_random_seed
tf.random.set_seed(10)

In [20]:
train_imgs_folder = './input/train_images/'

In [21]:
class DataGenenerator(Sequence):
    def __init__(self, images_list=None, folder_imgs=train_imgs_folder, 
                 batch_size=32, shuffle=True, augmentation=None,
                 resized_height=260, resized_width=260, num_channels=3):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augmentation = augmentation
        if images_list is None:
            self.images_list = os.listdir(folder_imgs)
        else:
            self.images_list = deepcopy(images_list)
        self.folder_imgs = folder_imgs
        self.len = len(self.images_list) // self.batch_size
        self.resized_height = resized_height
        self.resized_width = resized_width
        self.num_channels = num_channels
        self.num_classes = 4
        self.is_test = not 'train' in folder_imgs
        if not shuffle and not self.is_test:
            self.labels = [img_2_ohe_vector[img] for img in self.images_list[:self.len*self.batch_size]]

    def __len__(self):
        return self.len
    
    def on_epoch_start(self):
        if self.shuffle:
            random.shuffle(self.images_list)

    def __getitem__(self, idx):
        current_batch = self.images_list[idx * self.batch_size: (idx + 1) * self.batch_size]
        X = np.empty((self.batch_size, self.resized_height, self.resized_width, self.num_channels))
        y = np.empty((self.batch_size, self.num_classes))

        for i, image_name in enumerate(current_batch):
            path = os.path.join(self.folder_imgs, image_name)
            img = cv2.resize(cv2.imread(path), (self.resized_height, self.resized_width)).astype(np.float32)
            if not self.augmentation is None:
                augmented = self.augmentation(image=img)
                img = augmented['image']
            X[i, :, :, :] = img/255.0
            if not self.is_test:
                y[i, :] = img_2_ohe_vector[image_name]
        return X, y

    def get_labels(self):
        if self.shuffle:
            images_current = self.images_list[:self.len*self.batch_size]
            labels = [img_2_ohe_vector[img] for img in images_current]
        else:
            labels = self.labels
        return np.array(labels)

In [22]:
albumentations_train = Compose([
    VerticalFlip(), HorizontalFlip(), Rotate(limit=20), GridDistortion()
], p=1)

In [23]:
albumentations_train

Compose([
  VerticalFlip(always_apply=False, p=0.5),
  HorizontalFlip(always_apply=False, p=0.5),
  Rotate(always_apply=False, p=0.5, limit=(-20, 20), interpolation=1, border_mode=4, value=None, mask_value=None),
  GridDistortion(always_apply=False, p=0.5, num_steps=5, distort_limit=(-0.3, 0.3), interpolation=1, border_mode=4, value=None, mask_value=None),
], p=1, bbox_params=None, keypoint_params=None, additional_targets={})

In [24]:
data_generator_train = DataGenenerator(train_imgs, augmentation=albumentations_train)

In [25]:
data_generator_train

In [26]:
data_generator_train_eval = DataGenenerator(train_imgs, shuffle=False)
data_generator_val = DataGenenerator(val_imgs, shuffle=False)

In [27]:
data_generator_train_eval

In [28]:
data_generator_val

In [31]:
class PrAucCallback(Callback):
    def __init__(self, data_generator, num_workers=num_cores, 
                 early_stopping_patience=5, 
                 plateau_patience=3, reduction_rate=0.5,
                 stage='train', checkpoints_path='checkpoints/'):
        super(Callback, self).__init__()
        self.data_generator = data_generator
        self.num_workers = num_workers
        self.class_names = ['Fish', 'Flower', 'Sugar', 'Gravel']
        self.history = [[] for _ in range(len(self.class_names) + 1)] # to store per each class and also mean PR AUC
        self.early_stopping_patience = early_stopping_patience
        self.plateau_patience = plateau_patience
        self.reduction_rate = reduction_rate
        self.stage = stage
        self.best_pr_auc = -float('inf')
        if not os.path.exists(checkpoints_path):
            os.makedirs(checkpoints_path)
        self.checkpoints_path = checkpoints_path
        
    def compute_pr_auc(self, y_true, y_pred):
        pr_auc_mean = 0
        print(f"\n{'#'*30}\n")
        for class_i in range(len(self.class_names)):
            precision, recall, _ = precision_recall_curve(y_true[:, class_i], y_pred[:, class_i])
            pr_auc = auc(recall, precision)
            pr_auc_mean += pr_auc/len(self.class_names)
            print(f"PR AUC {self.class_names[class_i]}, {self.stage}: {pr_auc:.3f}\n")
            self.history[class_i].append(pr_auc)        
        print(f"\n{'#'*20}\n PR AUC mean, {self.stage}: {pr_auc_mean:.3f}\n{'#'*20}\n")
        self.history[-1].append(pr_auc_mean)
        return pr_auc_mean
              
    def is_patience_lost(self, patience):
        if len(self.history[-1]) > patience:
            best_performance = max(self.history[-1][-(patience + 1):-1])
            return best_performance == self.history[-1][-(patience + 1)] and best_performance >= self.history[-1][-1]    
              
    def early_stopping_check(self, pr_auc_mean):
        if self.is_patience_lost(self.early_stopping_patience):
            self.model.stop_training = True    
              
    def model_checkpoint(self, pr_auc_mean, epoch):
        if pr_auc_mean > self.best_pr_auc:
            # remove previous checkpoints to save space
            for checkpoint in glob.glob(os.path.join(self.checkpoints_path, 'classifier_epoch_*')):
                os.remove(checkpoint)
        self.best_pr_auc = pr_auc_mean
        self.model.save(os.path.join(self.checkpoints_path, f'classifier_epoch_{epoch}_val_pr_auc_{pr_auc_mean}.h5'))              
        print(f"\n{'#'*20}\nSaved new checkpoint\n{'#'*20}\n")
              
    def reduce_lr_on_plateau(self):
        if self.is_patience_lost(self.plateau_patience):
            new_lr = float(keras.backend.get_value(self.model.optimizer.lr)) * self.reduction_rate
            keras.backend.set_value(self.model.optimizer.lr, new_lr)
            print(f"\n{'#'*20}\nReduced learning rate to {new_lr}.\n{'#'*20}\n")
        
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict_generator(self.data_generator, workers=self.num_workers)
        y_true = self.data_generator.get_labels()
        # estimate AUC under precision recall curve for each class
        pr_auc_mean = self.compute_pr_auc(y_true, y_pred)
              
        if self.stage == 'val':
            # early stop after early_stopping_patience=4 epochs of no improvement in mean PR AUC
            self.early_stopping_check(pr_auc_mean)

            # save a model with the best PR AUC in validation
            self.model_checkpoint(pr_auc_mean, epoch)

            # reduce learning rate on PR AUC plateau
            self.reduce_lr_on_plateau()            
        
    def get_pr_auc_history(self):
        return self.history


In [30]:
test_imgs_folder = '../input/test_images/'
train_imgs_folder = '../input/train_images/'
num_cores = multiprocessing.cpu_count()

In [ ]:
num_cores

In [32]:
train_metric_callback = PrAucCallback(data_generator_train_eval)
val_callback = PrAucCallback(data_generator_val, stage='val')

In [33]:
train_metric_callback.reduction_rate

0.5

In [34]:
val_callback

In [35]:
from keras.losses import binary_crossentropy
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

In [ ]:
pip install -U git+https://github.com/qubvel/efficientnet

In [41]:
from keras_radam import RAdam

In [42]:
def get_model():
    base_model = DenseNet121(weights='imagenet', include_top=False, pooling='avg')
    x = base_model.output
    y_pred = Dense(4, activation='sigmoid')(x)
    return Model(inputs=base_model.input, outputs=y_pred)

In [43]:
model = get_model()


In [1]:


from keras_radam import RAdam


for base_layer in model.layers[:-3]:
    base_layer.trainable = False
    
model.compile(  loss='categorical_crossentropy', metrics=['accuracy'])
history_0 = model.fit_generator(generator=data_generator_train,
                              validation_data=data_generator_val,
                              epochs=20,
                              callbacks=[train_metric_callback, val_callback],
                              workers=num_cores,
                              verbose=1
                             )

Using TensorFlow backend.


NameError: name 'model' is not defined